# Imports

In [2]:
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler, AutoencoderKL
import torch
from utils import get_device, plot_img

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


# Constants

In [3]:
model_id = "stabilityai/stable-diffusion-2-1"
inf_steps = 20
device = get_device()
prompt = "portrait photo of a old warrior chief, tribal panther make up, blue on red, serious eyes, in front of forest 50mm portrait photography, hard rim lighting photography--beta --ar 2:3  --beta --upbeta"

# Create Model

In [4]:
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, safety_checker=None)
# Scheduler type - Euler has better quality, DPM is faster
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
#pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")
pipe.enable_attention_slicing()
vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse", torch_dtype=torch.float16).to("cuda")
pipe.vae = vae

Fetching 13 files: 100%|██████████| 13/13 [00:00<00:00, 135974.94it/s]
/home/shane/projects/.venv/lib/python3.10/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


# Baseline

In [5]:
generator = torch.Generator("cuda").manual_seed(60)   
image = pipe(prompt, generator=generator, num_inference_steps=inf_steps).images[0]
plot_img(image)

100%|██████████| 20/20 [00:11<00:00,  1.74it/s]


# cuDNN auto-tuner

In [ ]:
torch.backends.cudnn.benchmark = True
generator = torch.Generator("cuda").manual_seed(60)   
image = pipe(prompt, generator=generator, num_inference_steps=inf_steps).images[0]
plot_img(image)